In [22]:
import os
import pandas as pd

In [28]:
input_folder = "/content/drive/MyDrive/Company_Raw_Data/Infrastructure"
output_folder = "/content/drive/MyDrive/Cleaned Data/Infrastructure"
os.makedirs(output_folder, exist_ok=True)

In [29]:
# Sheet names to extract
sheet_names = ["Balance Sheet", "Cash Flow", "Profit & Loss", "Quarters"]

In [30]:
def clean_df(df):
    # Clean column names
    df.columns = (
        df.columns
        .astype(str)
        .str.replace(r'\s+', ' ', regex=True)
        .str.replace(r'[^\w\s]', '', regex=True)
        .str.strip()
    )

    # Clean index
    df.index = (
        df.index.astype(str)
        .str.replace(r'\s+', ' ', regex=True)
        .str.strip()
    )

    # Clean first column (typically labels)
    if df.shape[1] > 1:
        df.iloc[:, 0] = (
            df.iloc[:, 0].astype(str)
            .str.replace(r'\+', '', regex=True)
            .str.replace(r'%', '', regex=True)
            .str.replace(r'\s+', ' ', regex=True)
            .str.strip()
        )

    # Remove all % signs from entire DataFrame
    df = df.replace(r'%', '', regex=True)

    # Convert columns (except first) to numeric
    for col in df.columns[1:]:
        if not pd.api.types.is_numeric_dtype(df[col]):
            df[col] = pd.to_numeric(df[col], errors='coerce')


    # Convert row index into column temporarily
    df.reset_index(inplace=True)

    # Rename first column (previous row labels) to 'Micro_Data'
    df.rename(columns={'Unnamed 0': 'Financial_Metric'}, inplace=True)

    # Drop columns with more than 50% missing values
    df = df.dropna(thresh=0.5 * len(df), axis=1)

    # Restore the original index (first column becomes new index)
    df.set_index(df.columns[0], inplace=True)

    return df


In [31]:
# Process and save one Excel file
def process_excel_file(file_path):
    company_name = os.path.splitext(os.path.basename(file_path))[0]
    print(f"\n📂 Processing: {company_name}")

    try:
        xls = pd.ExcelFile(file_path)
        for sheet in sheet_names:
            if sheet in xls.sheet_names:
                df = pd.read_excel(xls, sheet_name=sheet)
                df = clean_df(df)

                # Save to CSV
                sheet_clean_name = sheet.replace(' ', '_')
                output_file = os.path.join(output_folder, f"{company_name}_{sheet_clean_name}.csv")
                df.to_csv(output_file, index=False)
                print(f"✅ Saved: {output_file}")

                # Display first 10 rows
                print(f"\n📄 Cleaned Sheet: {company_name} - {sheet}")
                print(df.head(10).to_string(index=False))  # Pretty print

    except Exception as e:
        print(f"❌ Error processing {file_path}: {e}")


In [32]:
# Loop through Excel files
for filename in os.listdir(input_folder):
    if filename.endswith(('.xlsx', '.xls')) and not filename.startswith('~$'):
        file_path = os.path.join(input_folder, filename)
        process_excel_file(file_path)


📂 Processing: LT
✅ Saved: /content/drive/MyDrive/Cleaned Data/Infrastructure/LT_Balance_Sheet.csv

📄 Cleaned Sheet: LT - Balance Sheet
 Financial_Metric  Mar 2014  Mar 2015  Mar 2016  Mar 2017  Mar 2018  Mar 2019  Mar 2020  Mar 2021  Mar 2022  Mar 2023  Mar 2024  Mar 2025
   Equity Capital       185       186       186       187       280       281       281       281       281       281       275       275
         Reserves     37526     40723     43994     50030     54623     62094     66442     75588     82127     89045     86084     97381
       Borrowings     80846     93630     88135     93954    107524    125555    143174    134629    125508    120650    116322    132409
Other Liabilities     50324     58727     61775     67400     80812     90106     96790     99597    111112    119745    136412    149460
Total Liabilities    168881    193266    194091    211571    243240    278036    306687    310095    319028    329722    339094    379524
     Fixed Assets     32397     3473